In [0]:
#  !pip install keras_efficientnets

In [0]:
# !git clone https://github.com/tensorflow/serving.git

In [0]:
# !cd serving

In [0]:
# !ls

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My Drive/datasets/logic_lyrics

/content/drive/My Drive/datasets/logic_lyrics


In [3]:
!ls

logic_lyrics.txt  weights-improvement-01-2.7537.hdf5


In [5]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pandas as pd
import nltk
import re

Using TensorFlow backend.


In [6]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
with open("logic_lyrics.txt","r") as f:
    text_data = f.read()

In [8]:
len(text_data)

317323

In [0]:
text_data = text_data.lower()

In [0]:
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text) 

In [0]:
replace_dict = {"(":"", ")":""}

In [0]:
text_data = multiple_replace(replace_dict, text_data)

In [0]:
chars = sorted(list(set(text_data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
n_chars = len(text_data)
n_vocab = len(char_to_int)

In [0]:
# words = text_data.split()
# words_to_int = dict((c,i) for i,c in enumerate(words))

In [0]:
# n_words = len(words)
# n_vocab_words = len(words_to_int)

In [17]:

# prepare the dataset of input to output pairs encoded as integers
seq_length_char = 100
dataX_char = []
dataY_char = []
for i in range(0, n_chars - seq_length_char, 1):
	seq_in_char = text_data[i:i + seq_length_char]
	seq_out_char = text_data[i + seq_length_char]
	dataX_char.append([char_to_int[char] for char in seq_in_char])
	dataY_char.append(char_to_int[seq_out_char])
n_patterns_char = len(dataX_char)
print ("Total Patterns: ", n_patterns_char)

Total Patterns:  316412


In [18]:

# # prepare the dataset of input to output pairs encoded as integers
# seq_length_words = 10
# dataX_words = []
# dataY_words = []
# for i in range(0, n_words - seq_length_words, 1):
# 	seq_in_words = words[i:i + seq_length_words]
# 	seq_out_words = words[i + seq_length_words]
# 	dataX_words.append([words_to_int[word] for word in seq_in_words])
# 	dataY_words.append(words_to_int[seq_out_words])
# n_patterns_words = len(dataX_words)
# print ("Total Patterns: ", n_patterns_words)

Total Patterns:  62877


In [0]:
# reshape X to be [samples, time steps, features]
X_char = numpy.reshape(dataX_char, (n_patterns_char, seq_length_char, 1))
# normalize
X_char = X_char / float(n_vocab)
# one hot encode the output variable
y_char = np_utils.to_categorical(dataY_char)

In [0]:
# # reshape X to be [samples, time steps, features]
# X_words = numpy.reshape(dataX_words, (n_patterns_words, seq_length_words, 1))
# # normalize
# X_words = X_words / float(n_vocab_words)
# # one hot encode the output variable
# y_words = np_utils.to_categorical(dataY_words)

In [24]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X_char.shape[1], X_char.shape[2])))#, return_sequences=True))
# model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y_char.shape[1], activation='relu'))
model.add(Dense(y_char.shape[1], activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 51)                13107     
_________________________________________________________________
dense_4 (Dense)              (None, 51)                2652      
Total params: 279,951
Trainable params: 279,951
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [27]:
model.fit(X_char, y_char, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
316412/316412 [==============================] - 240s 757us/step - loss: 2.6115

Epoch 00001: loss improved from 2.75506 to 2.61148, saving model to weights-improvement-01-2.6115.hdf5
Epoch 2/100
316412/316412 [==============================] - 238s 753us/step - loss: 2.5391

Epoch 00002: loss improved from 2.61148 to 2.53908, saving model to weights-improvement-02-2.5391.hdf5
Epoch 3/100
316412/316412 [==============================] - 241s 762us/step - loss: 2.4740

Epoch 00003: loss improved from 2.53908 to 2.47402, saving model to weights-improvement-03-2.4740.hdf5
Epoch 4/100
316412/316412 [==============================] - 239s 755us/step - loss: 2.4183

Epoch 00004: loss improved from 2.47402 to 2.41829, saving model to weights-improvement-04-2.4183.hdf5
Epoch 5/100
316412/316412 [==============================] - 239s 757us/step - loss: 2.3681

Epoch 00005: loss improved from 2.41829 to 2.36811, saving model to weights-improvement-05-2.3681.hdf5
Epoch 6/100
316412/3

In [0]:
model.load_weights("weights-improvement-51-1.7654.hdf5")

In [0]:
import sys

In [34]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

# pick a random seed
start = numpy.random.randint(0, len(dataX_char)-1)
pattern = dataX_char[start]
#	print "Seed:"
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

" ey would never hate it, i'm a renegade it
say they love me, say they want me
say they bump me in the "
 soued if to be and get tp and ree and she sacond i got the seil to be alack i'm boing' io the sone
that i got the mine the sooe that i got the mine the thit then i get it i'm bacl ao you bnt words then i medd to the soae if so this soig it and go the seal gor so do it wotr back boonne wouls the sone that i got the mine the sooe that i got the mine the sooe that i got the mine the thit then i get it i'm bacl ao yhat
i'me never ba teere i mever be the reme to feeling like thes i mever ba the realod i wes a fronne woudd i was like that she sole the shit that i got the mine the sooe that i got the mene i don't know what i mever ba the sone that i got the mene i was aod the sone that i got the mene i don't know what i mever ba the sone that i got the mene i was aod the sone that i got the mene i don't know what i mever ba the sone that i got the mene i was aod the sone that i got the me

In [0]:
model.save("model.h5")

In [36]:
!ls


logic_lyrics.txt		    weights-improvement-21-1.9644.hdf5
model_1.7654.h5			    weights-improvement-22-1.9537.hdf5
weights-improvement-01-2.6115.hdf5  weights-improvement-23-1.9452.hdf5
weights-improvement-01-2.7537.hdf5  weights-improvement-24-1.9330.hdf5
weights-improvement-01-2.9581.hdf5  weights-improvement-25-1.9190.hdf5
weights-improvement-02-2.5391.hdf5  weights-improvement-26-1.9107.hdf5
weights-improvement-02-2.7551.hdf5  weights-improvement-27-1.8999.hdf5
weights-improvement-03-2.4740.hdf5  weights-improvement-28-1.8901.hdf5
weights-improvement-04-2.4183.hdf5  weights-improvement-29-1.8829.hdf5
weights-improvement-05-2.3681.hdf5  weights-improvement-30-1.8741.hdf5
weights-improvement-06-2.3232.hdf5  weights-improvement-31-1.8673.hdf5
weights-improvement-07-2.2848.hdf5  weights-improvement-33-1.8663.hdf5
weights-improvement-08-2.2499.hdf5  weights-improvement-34-1.8565.hdf5
weights-improvement-09-2.2176.hdf5  weights-improvement-35-1.8563.hdf5
weights-improvement-10-2.1882.hdf5

51